In [5]:
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
import numpy as np
import os

# Función para extraer texto del HTML basado en etiquetas específicas.
def extract_text_from_html(html, tags=['title', 'meta', 'body']):
        soup = BeautifulSoup(html, 'html.parser')
        texts = []
        for tag in tags:
            if tag == 'meta':
                for meta in soup.find_all('meta', attrs={"name": "description"}):
                    texts.append(meta.get('content', ''))
            else:
                element = soup.find(tag)
                if element:
                    texts.append(element.get_text())
        return ' '.join(texts)
        
folder_path = "../data/splits/train"
for dirs in os.listdir(folder_path):
    # Supongamos que html_contents es una lista de documentos HTML.
    html_contents = [os.path.join(folder_path, dirs, filename) for filename in os.listdir(os.path.join(folder_path, dirs))]  # Aquí iría tu lista de HTML.
    #print(html_contents)
    # Preprocesamiento y vectorización del texto.
    pipeline = Pipeline([
        ('vect', CountVectorizer(stop_words='english')),
        ('tfidf', TfidfTransformer()),
        ('chi', SelectKBest(chi2, k=1000)),  # k es el número de características que deseas mantener.
    ])
    
    # Extraer texto de cada HTML y almacenarlo en una lista.
    text_data = [extract_text_from_html(html) for html in html_contents]
    print(text_data)
    # Ajustar el pipeline a tus datos.
    X_features = pipeline.fit_transform(text_data, y=None)  # y serían tus etiquetas si estás haciendo aprendizaje supervisado.
    
    # Ahora X_features contiene las características extraídas listas para ser usadas en un clasificador.
    print(X_features.shape)


<ipython-input-5-dde4b7b96977>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from bs4 import BeautifulSoup, Comment

# Función para obtener la profundidad máxima del árbol DOM
def get_dom_tree_depth(element, depth=0):
    if hasattr(element, 'children'):
        return max((get_dom_tree_depth(child, depth + 1) for child in element.children if child.name is not None), default=depth)
    return depth

# Función para extraer características de un contenido HTML
def extract_html_features(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Obtener el título de la página, si está presente
    title = soup.title.string if soup.title else ''
    
    # Obtener todas las metatags y sus contenidos
    meta_tags = {meta.attrs.get('name', ''): meta.attrs.get('content', '') for meta in soup.find_all('meta')}
    
    # Obtener todos los encabezados y su texto
    headers = {f'h{i}': [header.text for header in soup.find_all(f'h{i}')] for i in range(1, 7)}
    
    # Obtener todos los párrafos y su texto
    paragraphs = [p.text for p in soup.find_all('p')]
    
    # Obtener todos los enlaces y sus hrefs
    links = [a['href'] for a in soup.find_all('a', href=True)]
    
    # Obtener todas las imágenes y sus atributos src y alt
    images = [{'src': img['src'], 'alt': img.get('alt', '')} for img in soup.find_all('img')]
    
    # Obtener todos los formularios
    forms = [form for form in soup.find_all('form')]
    
    # Contar todos los scripts y hojas de estilo
    scripts = len(soup.find_all('script'))
    stylesheets = len(soup.find_all('link', rel='stylesheet'))
    
    # Obtener etiquetas semánticas y su contenido
    semantic_tags = {tag: [t.text for t in soup.find_all(tag)] for tag in ['article', 'section', 'aside', 'header', 'footer']}
    
    # Obtener todos los comentarios
    comments = [str(comment) for comment in soup.find_all(string=lambda text: isinstance(text, Comment))]
    
    # Obtener todos los datos estructurados en formato JSON-LD
    structured_data = [script.text for script in soup.find_all('script', type='application/ld+json')]
    
    # Calcular la profundidad del árbol DOM
    dom_tree_depth = get_dom_tree_depth(soup)
    
    # Realizar análisis de frecuencia de palabras clave
    text = soup.get_text().lower()
    words = text.split()
    keyword_frequency = {word: words.count(word) for word in set(words)}
    
    # Compilar todas las características en un diccionario
    features = {
        'title': title,
        'meta': meta_tags,
        'headers': headers,
        'paragraphs': paragraphs,
        'links': links,
        'images': images,
        'forms': forms,
        'scripts': scripts,
        'stylesheets': stylesheets,
        'semantic_tags': semantic_tags,
        'comments': comments,
        'structured_data': structured_data,
        'dom_tree_depth': dom_tree_depth,
        'keyword_frequency': keyword_frequency
    }
    
    return features

# Ejemplo de cómo usar la función con contenido HTML
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Test HTML</title>
    <meta name="description" content="This is a test HTML file">
    <meta name="keywords" content="test, HTML, file">
</head>
<body>
    <h1>Main Heading</h1>
    <p>This is a paragraph in the body.</p>
    <a href="http://example.com">Example link</a>
</body>
</html>
"""

# Extraer características del contenido HTML de ejemplo
features = extract_html_features(html_content)
print(features)


In [9]:
#!pip3 install keybert

In [55]:
# First, let's write a Python script that uses BeautifulSoup, a popular web scraping library,
# to extract the features mentioned from an HTML page.

from bs4 import BeautifulSoup, Comment
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from keybert import KeyBERT

def get_dom_tree_depth(element, depth=0):
    if hasattr(element, 'children'):
        return max((get_dom_tree_depth(child, depth + 1) for child in element.children if child.name is not None), default=depth)
    return depth
def get_sumary(texts,sentences_count=5):
    # Extract the text from the page's body
    page_text = ' '.join(texts)
    # Perform summarization using sumy
    parser = PlaintextParser.from_string(page_text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count=sentences_count)  # You can adjust the number of sentences in the summary.
    #print(summary)
    return ' '.join([str(sentence) for sentence in summary])
# Calculate the DOM tree depth (recursively)
def get_dom_tree_depth(element, depth=0):
        return max([get_dom_tree_depth(child, depth + 1) for child in element.children if child.name is not None], default=depth)
# This function will take HTML content as an input and return extracted features.
def extract_html_features(html_content,label='unknow'):
    modelo = KeyBERT()
    with open(html_content, 'r', encoding='latin-1') as file:
        text = file.read()
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    
    max_link=3
    # Initialize a dictionary to store our features
    features = {
        'title': soup.title.string if soup.title else '',
        'summary': get_sumary(soup.stripped_strings,sentences_count=2),
        'meta': {meta.attrs['name']: meta.attrs.get('content', '') for meta in soup.find_all('meta', attrs={'name': True})},
        'headers': {f'h{i}': [header.text for header in soup.find_all(f'h{i}')] for i in range(1, 3)},
        'paragraphs': [p.text for p in soup.find_all('p')],
        'links': [a['href'] for a in soup.find_all('a', href=True)][:max_link],
        'lists': {
            'unordered': get_sumary([ul.text for ul in soup.find_all('ul')]),
            'ordered': [ol.text for ol in soup.find_all('ol')]
        },
        'images': [{'src': img.get('src', ''), 'alt': img.get('alt', '')} for img in soup.find_all('img')],
        'forms':  [{'action': form.get('action'), 'method': form.get('method')} for form in soup.find_all('form')],
        'scripts': len(soup.find_all('script')),
        'stylesheets': len(soup.find_all('link', rel='stylesheet')),
        'semantic_tags': {tag: [str(t.text) for t in soup.find_all(tag)] for tag in ['article', 'section', 'aside', 'header', 'footer']},
        'comments': len([comment for comment in soup.find_all(string=lambda text: isinstance(text, Comment))]),
        'url_structure': '',  # Placeholder for URL, should be filled with the actual URL structure
        'structured_data': [script.text for script in soup.find_all('script', type='application/ld+json')],
        'dom_tree_depth': '',  # Placeholder for DOM tree depth calculation
        'keyword_frequency': {},  # Placeholder for keyword frequency analysis
        'keyword_frequency_kebert': {},  # Placeholder for keyword frequency analysis
        'ground_truth':label
    }
    
    features['dom_tree_depth'] = get_dom_tree_depth(soup)

    # Perform keyword frequency analysis
    text = soup.get_text().lower()
    words = text.split()
    # Lista de stop words en inglés (puedes extenderla o modificarla según sea necesario)
    stop_words = set(['the', 'and', 'is', 'in', 'to', 'a', 'of', 'for', 'on', 'that', 'with', 'as', 'it', 'at', 'by', 'from', 'this', 'an', 'be', 'are', 'was', 'or', 'which', 'has', 'you', 'your', 'have', 'but', 'not', 'we', 'they', 'will', 'its', 'their', 'all', 'can', 'more', 'any', 'if', 'my', 'our', 'about', 'may', 'such', 'other', 'into', 'than', 'these', 'her', 'would', 'him', 'his', 'some', 'what', 'over', 'also', 'when', 'out', 'there', 'after', 'first', 'up', 'one', 'last', 'new', 'no', 'most', 'been', 'who', 'its'])
    
    # Calcular la frecuencia de palabras sin contar las stop words
    words = [word for word in text.split() if word not in stop_words]
    keyword_frequency = {word: words.count(word) for word in set(words)}
    palabras_clave = modelo.extract_keywords(text, stop_words='english')
    features['keyword_frequency_kebert'] = [palabra for palabra,_ in palabras_clave]
    N=10
    features['keyword_frequency'] = list(dict(sorted(keyword_frequency.items(), key=lambda item: item[1], reverse=True)[:N]).keys())

    return features

# Example usage (assuming 'html_content' is a variable containing the HTML source code):
# html_features = extract_html_features(html_content)
# print(html_features)

# Since we cannot scrape a live webpage, we would need HTML content provided to utilize this function.
# If you have an HTML file or string you would like to analyze, please provide it.


In [56]:
extract_html_features('../data/splits/train/course/achmly')

{'title': 'ECE/CS 752 Spring 1996',
 'summary': 'Date: Mon, 11 Nov 1996 16:02:43 GMT Server: NCSA/1.5.2 Last-modified: Fri, 10 May 1996 15:09:04 GMT Content-type: text/html Content-length: 7346 ECE/CS 752 Spring 1996 ECE/CS 752: Advanced Computer Architecture I Spring 1996 Offering Course Information Instructor: Prof. James E. Smith Office:         4621 Engineering Hall Office hours:   2:30-3:30PM, Tues. and Thurs. Office phone:   265-5737 Email: jes@ece.wisc.edu TA: Dan Prince Office:         3614 Engineering Hall Office hours:   2:00-3:00PM Wed; 1:00-2:00PM Fri Office phone:   265-3825 E-mail address: princed@cae.wisc.edu Table of Contents News Readings Lecture Notes Homeworks Project Miscellaneous News Homework 5 solns  5/10 Special Office hours: 10-11:30AM Fri. May 10 FINAL EXAM: Rm 132 Biochemistry, Sun.',
 'meta': {},
 'headers': {'h1': ['ECE/CS 752: Advanced Computer Architecture I ',
   'Spring 1996 Offering'],
  'h2': [' Table of Contents',
   'News',
   'Readings',
   ' Lectu

In [57]:
import tqdm
import json
# Folder path containing the original files
#folder_path = "C:/Users/user/Downloads/dataset_splitted/splits/test"
folder_path = "../data/splits/train"

# Output folder path for JSON files
#output_folder = "C:/Users/user/Downloads/dataset_splitted/splits/test_json"
output_folder = "../data/splits/train_new_data/"

os.makedirs(output_folder, exist_ok=True)
# Process all files in the folder
for dirs in os.listdir(folder_path):
    for filename in tqdm.tqdm(os.listdir(os.path.join(folder_path, dirs)), desc="Extrayendo datos"):
        file_path = os.path.join(folder_path, dirs, filename)
        #print(file_path)
        if os.path.isfile(file_path):
            formatted_data = extract_html_features(file_path,dirs)
            #print(formatted_data)
            #print("Processed:", filename)
            #print("Title:", formatted_data["Title"])
            #print("Summary:", formatted_data["Body"])
            #print("Ground Truth:", formatted_data["ground_truth"])
            # Save the data to a JSON fileç
            output_folder_dir=os.path.join(output_folder,dirs)
            os.makedirs(output_folder_dir, exist_ok=True)
            output_json_file = os.path.join(output_folder_dir, os.path.splitext(filename)[0] + ".json")
            with open(output_json_file, 'w', encoding='utf-8') as json_file:
                json.dump(formatted_data, json_file, ensure_ascii=False, indent=4)

Extrayendo datos:  10%|██▎                   | 309/3011 [01:16<09:05,  4.95it/s]<ipython-input-55-dc2b7d3831da>:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
Extrayendo datos: 100%|███████████████████████| 744/744 [02:51<00:00,  4.33it/s]


In [59]:
import tqdm
import json
# Folder path containing the original files
#folder_path = "C:/Users/user/Downloads/dataset_splitted/splits/test"
folder_path = "../data/splits/test"

# Output folder path for JSON files
#output_folder = "C:/Users/user/Downloads/dataset_splitted/splits/test_json"
output_folder = "../data/splits/test_new_data/test"

os.makedirs(output_folder, exist_ok=True)
# Process all files in the folder
for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        #print(file_path)
        if os.path.isfile(file_path):
            formatted_data = extract_html_features(file_path)
            #print(formatted_data)
            #print("Processed:", filename)
            #print("Title:", formatted_data["Title"])
            #print("Summary:", formatted_data["Body"])
            #print("Ground Truth:", formatted_data["ground_truth"])
            # Save the data to a JSON fileç
            output_folder_dir=os.path.join(output_folder,dirs)
            os.makedirs(output_folder_dir, exist_ok=True)
            output_json_file = os.path.join(output_folder_dir, os.path.splitext(filename)[0] + ".json")
            with open(output_json_file, 'w', encoding='utf-8') as json_file:
                json.dump(formatted_data, json_file, ensure_ascii=False, indent=4)

In [ ]:

folder_path = "../data/splits/train"
for dirs in os.listdir(folder_path):
    # Supongamos que html_contents es una lista de documentos HTML.
    html_contents = [os.path.join(folder_path, dirs, filename) for filename in os.listdir(os.path.join(folder_path, dirs))]  # Aquí iría tu lista de HTML.
    #print(html_contents)
    # Preprocesamiento y vectorización del texto.
    
    # Extraer texto de cada HTML y almacenarlo en una lista.
    text_data = [extract_html_features(html) for html in html_contents]
    print(text_data)
    # Ajustar el pipeline a tus datos.
    # Ahora X_features contiene las características extraídas listas para ser usadas en un clasificador.


In [ ]:
from collections import Counter
from urllib.parse import urlparse

def extract_links(soup):
    """ Extrae todos los enlaces y sus textos de un objeto BeautifulSoup. """
    links = []
    for a_tag in soup.find_all('a', href=True):
        url = a_tag['href']
        text = a_tag.get_text(strip=True)
        links.append((url, text))
    return links

def get_most_representative_links(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = extract_links(soup)

    # Contar frecuencia de cada enlace
    link_freq = Counter(links)

    # Filtrar por enlaces con texto descriptivo
    descriptive_links = [(url, text) for url, text in links if len(text) > 3]

    # Contar la frecuencia de los enlaces descriptivos
    descriptive_link_freq = Counter(descriptive_links)

    # Podrías ordenar por frecuencia y tomar los N enlaces más frecuentes o descriptivos
    N = 20
    most_common_links = descriptive_link_freq.most_common(N)

    return most_common_links

folder_path = "../data/splits/train/course"

for dirs in os.listdir(folder_path):
    #extract_html_features()
    with open(os.path.join(folder_path, dirs), 'r', encoding='latin-1') as file:
            html_content = file.read()
    # Usar la función con tu contenido HTML
    most_representative_links = get_most_representative_links(html_content)
    print(most_representative_links)


In [ ]:
from bs4 import BeautifulSoup
from collections import Counter
import os

def extract_links_html(soup):
    """ Extrae todos los enlaces y sus textos de un objeto BeautifulSoup. """
    links = []
    for a_tag in soup.find_all('a', href=True):
        url = a_tag['href']
        text = a_tag.get_text(strip=True)
        links.append((url, text))
    return links

def extract_links(html_content):
    """ Extrae todos los enlaces (href) de un objeto BeautifulSoup. """
    soup = BeautifulSoup(html_content, 'html.parser')
    return extract_links_html(soup)

def find_common_urls(html_documents):
    """ Encuentra los URLs más comunes en una lista de documentos HTML. """
    all_links = []
    
    # Extraer enlaces de cada documento HTML
    for html_content in html_documents:
        links = extract_links(html_content)
        all_links.extend(links)
    
    # Contar la frecuencia de cada enlace
    link_counter = Counter(all_links)
    
    # Obtener los enlaces más comunes
    most_common_links = link_counter.most_common()
    
    return most_common_links


folder_path = "../data/splits/train/course"
# Suponiendo que tienes una lista de contenido HTML en html_contents
# Aquí tendrías que cargar tus documentos HTML en la lista html_contents
html_contents = []
# Listar todos los archivos en el directorio
for file_name in os.listdir(folder_path):
    # Construir la ruta completa al archivo
    file_path = os.path.join(folder_path, file_name)
    
    
        # Abrir y leer el contenido del archivo
    with open(file_path, 'r', encoding='latin-1') as file:
            html_content = file.read()
            html_contents.append(html_content)
            
print(html_contents)
# Encontrar y mostrar los URLs más comunes
common_urls = find_common_urls(html_contents)
for url, count in common_urls:
    print(f"URL: {url}, Frecuencia: {count} len: {len(html_contents)}")


In [ ]:
from bs4 import BeautifulSoup
import os
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descargar el conjunto de stop words si aún no se ha hecho (se hace una sola vez)
nltk.download('punkt')
nltk.download('stopwords')

# Cargar las stop words en inglés
stop_words = set(stopwords.words('english'))

def clean_html(html_content):
    """ Elimina todas las etiquetas HTML y devuelve solo el texto. """
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.get_text(separator=' ')
    return text

def get_most_representative_words(html_documents):
    """ Encuentra las palabras más representativas en una lista de documentos HTML. """
    all_words = []

    # Procesar cada documento HTML
    for html_content in html_documents:
        text = clean_html(html_content)
        # Convertir a minúsculas y tokenizar
        words = word_tokenize(text.lower())
        # Eliminar stop words y palabras que no sean alfabéticas
        words = [word for word in words if word.isalpha() and word not in stop_words]
        all_words.extend(words)
    
    # Contar la frecuencia de cada palabra
    word_counter = Counter(all_words)
    
    # Obtener las palabras más comunes
    most_common_words = word_counter.most_common()
    
    return most_common_words

# Supongamos que ya tienes una lista de contenido HTML llamada html_contents
# Aquí iría el código para cargar tus documentos HTML en la lista html_contents

# Encontrar y mostrar las palabras más representativas
representative_words = get_most_representative_words(html_contents)
for word, count in representative_words:
    print(f"Palabra: {word}, Frecuencia: {count}")


In [ ]:
modelo = KeyBERT()
modelo.extract_keywords(html_contents, stop_words='english')